In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from functools import partial
from pathlib import Path

import numpy as np
import pandas as pd
import torch as T
import torch.optim as optim

In [3]:
cd ..

C:\Projects\python\recommender


In [4]:
from datasets import TorchMovielen10k, TorchTopcoder, DataBunch
from models import FMLearner, TorchFM, TorchHrmFM, TorchPrmeFM, TorchTransFM
from models.fm_learner import simple_loss, trans_loss

In [5]:
DEVICE = T.cuda.current_device()
BATCH = 2000
SHUFFLE = True
WORKERS = 0
REGS_PATH = Path("./inputs/topcoder/regs.csv")

In [6]:
topcoder_db = TorchTopcoder(REGS_PATH)
topcoder_db

2019-09-12 14:44:32,538 - C:\Projects\python\recommender\utils.py - INFO - Read dataset in inputs\topcoder\regs.csv
2019-09-12 14:44:32,538 - C:\Projects\python\recommender\utils.py - INFO - Read dataset in inputs\topcoder\regs.csv
2019-09-12 14:44:32,538 - C:\Projects\python\recommender\utils.py - INFO - Read dataset in inputs\topcoder\regs.csv
I0912 14:44:32.538454 14248 torch_topcoder.py:47] Read dataset in inputs\topcoder\regs.csv
2019-09-12 14:44:32,545 - C:\Projects\python\recommender\utils.py - INFO - Original regs shape: (610025, 3)
2019-09-12 14:44:32,545 - C:\Projects\python\recommender\utils.py - INFO - Original regs shape: (610025, 3)
2019-09-12 14:44:32,545 - C:\Projects\python\recommender\utils.py - INFO - Original regs shape: (610025, 3)
I0912 14:44:32.545405 14248 torch_topcoder.py:48] Original regs shape: (610025, 3)
2019-09-12 14:44:32,638 - C:\Projects\python\recommender\utils.py - INFO - Original registants size: 60017
2019-09-12 14:44:32,638 - C:\Projects\python\re

In [7]:
topcoder_db.config_db(batch_size=BATCH,
                      shuffle=SHUFFLE,
                      num_workers=WORKERS,
                      device=DEVICE)

In [8]:
feat_dim = topcoder_db.feat_dim
NUM_DIM = 124
INIT_MEAN = 0.1

### Create criterion

In [9]:
# regst setting
LINEAR_REG = 1
EMB_REG = 1
TRANS_REG = 1

In [10]:
simple_loss_callback = partial(simple_loss, LINEAR_REG, EMB_REG)
simple_loss_callback

functools.partial(<function simple_loss at 0x0000021FA683D2F0>, 1, 1)

### Train model

In [11]:
LEARNING_RATE = 0.001
DECAY_FREQ = 1000
DECAY_GAMMA = 1

In [12]:
fm_model = TorchFM(feature_dim=feat_dim, num_dim=NUM_DIM, init_mean=INIT_MEAN)
fm_model

TorchFM()

In [13]:
adam_opt = optim.Adam(fm_model.parameters(), lr=LEARNING_RATE)
schedular = optim.lr_scheduler.StepLR(adam_opt, step_size=1000, gamma=DECAY_GAMMA)

In [14]:
fm_learner = FMLearner(fm_model, adam_opt, schedular, topcoder_db)
fm_learner

In [18]:
fm_learner.fit(epoch=1, loss_callback=simple_loss_callback)

  0%|                                                                                                                                                                                  | 0/1 [00:00<?, ?it/s]

Epoch: 0
Epoch 0 step 0: training loss: 36595.257463009104
Epoch 0 step 1: training loss: 35559.40638101382
Epoch 0 step 2: training loss: 34565.72414944783
Epoch 0 step 3: training loss: 33598.092547919754
Epoch 0 step 4: training loss: 32690.78531390714
Epoch 0 step 5: training loss: 31745.331099249604
Epoch 0 step 6: training loss: 30880.821221724556
Epoch 0 step 7: training loss: 30010.142860921136
Epoch 0 step 8: training loss: 29126.861485908918
Epoch 0 step 9: training loss: 28326.42653949232
Epoch 0 step 10: training loss: 27514.757163635364
Epoch 0 step 11: training loss: 26748.280416624504
Epoch 0 step 12: training loss: 25935.456451214493
Epoch 0 step 13: training loss: 25204.01370157827
Epoch 0 step 14: training loss: 24507.248573880766
Epoch 0 step 15: training loss: 23782.009283290605
Epoch 0 step 16: training loss: 23076.729034108594
Epoch 0 step 17: training loss: 22448.71968973585
Epoch 0 step 18: training loss: 21751.941589665737
Epoch 0 step 19: training loss: 21151.

Epoch 0 step 161: training loss: 471.59817159168153
Epoch 0 step 162: training loss: 454.55296137684695
Epoch 0 step 163: training loss: 441.30340475992307
Epoch 0 step 164: training loss: 459.97532873326094
Epoch 0 step 165: training loss: 425.96280588887595
Epoch 0 step 166: training loss: 450.1372598482842
Epoch 0 step 167: training loss: 438.6853764057085
Epoch 0 step 168: training loss: 439.2942222913774
Epoch 0 step 169: training loss: 420.39482700813556
Epoch 0 step 170: training loss: 422.9234605200323
Epoch 0 step 171: training loss: 415.51394037446073
Epoch 0 step 172: training loss: 429.7578772458437
Epoch 0 step 173: training loss: 428.0315576071465
Epoch 0 step 174: training loss: 415.13823235984836
Epoch 0 step 175: training loss: 400.3667745897594
Epoch 0 step 176: training loss: 402.7374074812492
Epoch 0 step 177: training loss: 391.6504437480455
Epoch 0 step 178: training loss: 395.3935424855055
Epoch 0 step 179: training loss: 400.18345007642887
Epoch 0 step 180: trai

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [01:49<00:00, 109.06s/it]
